In [1]:
import pyautogui as gui
gui.PAUSE = 1
import numpy as np
gui.FAILSAFE = True
import random
import time
import mouse as ms
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from collections import namedtuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from torch.autograd import variable

In [10]:
game = gui.getWindow('BlueStacks')
pos = game.get_position()
#possible_moves = np.identity(5,dtype='int').tolist()
possible_moves = [0,1,2,3,4]
possible_moves

[0, 1, 2, 3, 4]

In [11]:
pos

(61, 16, 429, 740)

In [12]:
ln = pos[2] - pos[0]
ht = pos[3] - pos[1]


In [13]:
replay_x = pos[0]+np.round(ln*0.7185)
replay_y = pos[1]+np.round(ht*0.86123)
replay_rect_x = pos[0]
replay_rect_y = pos[1]+np.round(ht*0.81598)
replay_rect_ln = ln
replay_rect_ht = np.round(ht*0.12)
replay_image = gui.grab(region=(replay_rect_x,replay_rect_y,replay_rect_ln,replay_rect_ht))

In [41]:
popup_x = pos[0] + np.round(ln*0.0479)
popup_y = pos[1] + np.round(ht*0.22322)
popup_ln = np.round(ln*0.1197)
popup_ht = np.round(ht*0.06033)
#popup_image = gui.grab(region=(popup_x,popup_y,popup_ln,popup_ht))
popup_click_x = pos[0] + np.round(ln*0.107844)
popup_click_y = pos[1] + np.round(ht*0.248868)

In [42]:
popup_image = np.load('popup_image_array.npy')

In [43]:
def isdone():
    replay_image_check = gui.grab(region=(replay_rect_x,replay_rect_y,replay_rect_ln,replay_rect_ht))
    if replay_image_check == replay_image:
        return True
    else:
        return False

def popup_check_close():
    popup_image_check = gui.grab(region=(popup_x,popup_y,popup_ln,popup_ht))
    if np.array_equal(np.array(popup_image_check),popup_image):
        ms.move(popup_click_x,popup_click_y)
        ms.click()

def replay():
    replay_state = cv2.resize(cv2.cvtColor(np.array(gui.grab(region=(pos[0],pos[1],ln,ht))), cv2.COLOR_BGR2GRAY),(60,110))
    ms.move(replay_x,replay_y)
    ms.click()
    return replay_state

In [44]:
popup_check_close()

In [17]:
def step(i):
    reward = 0
    image_before = cv2.resize(cv2.cvtColor(np.array(gui.grab(region=(pos[0],pos[1],ln,ht))), cv2.COLOR_BGR2GRAY),(60,110))
    if i == possible_moves[0]:
        game.set_foreground()
        gui.press('up')
        
    if i == possible_moves[1]:
        game.set_foreground()
        gui.press('down')
        
    if i == possible_moves[2]:
        game.set_foreground()
        gui.press('left')
        
    if i == possible_moves[3]:
        game.set_foreground()
        gui.press('right')
        
    if i == possible_moves[4]:
        game.set_foreground()
        pass
        
    image_after = cv2.resize(cv2.cvtColor(np.array(gui.grab(region=(pos[0],pos[1],ln,ht))), cv2.COLOR_BGR2GRAY),(60,110))
    if isdone() == False:
        reward = 1
    else:
        reward = 0
    return image_before,reward,isdone(),image_after
    

In [18]:
input_size = 1
output_size = 5
class Net(nn.Module):
    def __init__(self,input_size,output_size):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(input_size,32,kernel_size=3,stride=2,padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=2,padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=2,padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, stride=2,padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        self.linear = nn.Linear(896,output_size)
    
    def forward(self,x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        return self.linear(x.view(x.size(0), -1))



In [45]:
net = Net(input_size,output_size)
net = net.double()
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
epsilon = 0.1
while True:
    sm = nn.Softmax(dim=1)
    state = replay()
    reward = 0
    episode_reward = 0
    training_obs = []
    training_acts = []
    while True:
        move_probs = sm(net(torch.from_numpy(state/255).reshape(-1,1,110,60))).tolist()[0]
        move = np.random.choice(list(range(len(possible_moves))),p=move_probs)
        move = possible_moves[move]
#        move = np.random.choice([move,np.abs(1-move)],p=[1-epsilon,epsilon])
        state,reward,is_done,state_after = step(move)
        episode_reward += reward
        training_obs.append(state)
        training_acts.append(move)
        popup_check_close()
        if isdone():
            ms.release()
            training_obs = training_obs[0:-3]
            training_acts = training_acts[0:-3]
            
            if len(training_acts) > 100:
                print('training')
                training_obs_v = torch.from_numpy(np.array(training_obs)/255)
                training_acts_v = torch.from_numpy(np.array(training_acts))
                optimizer.zero_grad()
                action_scores_v = net(training_obs_v.reshape(-1,1, 110, 60))
                loss_v = objective(action_scores_v, variable(training_acts_v).long())
                loss_v.backward()
                optimizer.step()
                training_obs = []
                training_acts = []
                print('loss '+ str(loss_v.item()) + ' episode_reward '+ str(episode_reward))
                episode_reward = 0
            replay()
    if episode_reward > 1000:
        break

training


C:\ProgramData\Anaconda3\lib\site-packages\torch\autograd\__init__.py:167: UserWarning: torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead
  warnings.warn("torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead")


loss 1.5996184107290472 episode_reward 149
training
loss 0.8106094318168873 episode_reward 121
training
loss 0.4015357714739215 episode_reward 141
training
loss 0.1187684505978921 episode_reward 146
training
loss 0.008081359690448412 episode_reward 154
training
loss 0.0008185203777135877 episode_reward 176
training
loss 7.363385221206393e-05 episode_reward 119
training
loss 9.525862660340258e-06 episode_reward 116
training
loss 2.4274463687204023e-06 episode_reward 108
training
loss 1.0195589339356726e-06 episode_reward 116


KeyboardInterrupt: 

In [46]:
training_acts

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4]